# Generate Network Structure From .xlsx File

In [21]:
#import libraries 
import pandas as pd
import os 
import matplotlib.pyplot as plt
import numpy as np

In [22]:
#read Excel File 
os.ch
x = pd.read_excel('NationalOutbreakPublicDataTool.xlsx')

In [23]:
#make TWO new dataframes - one for norovirus (x_n) and one for salmonella (x_s)
norovirus = ['Norovirus Genogroup I', 'Norovirus Genogroup II','Norovirus unknown','Norovirus']
salmonella = ['Salmonella enterica']

#make norovirus dataframe
x_n = pd.DataFrame()
for i in norovirus:
    x_sub = x[x.Etiology == i]
    if i == 'Norovirus Genogroup I':
        x_n = x_sub
    else: x_n = pd.concat([x_ns, x_sub])

#make salmonella dataframe
x_s = pd.DataFrame()
x_s = x[x.Etiology == salmonella[0]]

x_n = x_n.reset_index()
x_s = x_s.reset_index()

In [24]:
#print preview of norovirus file 
x_n

,index,Year,Month,State,Primary Mode,Etiology,Serotype or Genotype,Etiology Status,Setting,Illnesses,...,Deaths,Info on Deaths,Food Vehicle,Food Contaminated Ingredient,IFSAC Category,Water Exposure,Water Type,Animal Type,Animal Type Specify,Water Status
0,31,2009,1,Tennessee,Food,Norovirus Genogroup I,NaN,Confirmed,Grocery store,8,...,0.0,8.0,cake,NaN,Multiple,NaN,NaN,NaN,NaN,NaN
1,32,2009,3,Tennessee,Food,Norovirus Genogroup I,NaN,Confirmed,Restaurant - Sit-down dining,9,...,0.0,9.0,"oysters, raw",oysters,Mollusks,NaN,NaN,NaN,NaN,NaN
2,41,2009,1,Tennessee,Food,Norovirus Genogroup I,NaN,Confirmed,"Restaurant - ""Fast-food""(drive up service or p...",6,...,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43,2009,1,Florida,Food,Norovirus Genogroup I,NaN,Confirmed,Restaurant - other or unknown type,24,...,0.0,24.0,"sandwich, wrap; sandwich, wrap",NaN,Multiple,NaN,NaN,NaN,NaN,NaN
4,79,2009,2,Ohio,Food,Norovirus Genogroup I,NaN,Confirmed,Caterer (food prepared off-site from where ser...,70,...,0.0,70.0,egg salad/egg salad sandwich,NaN,Multiple,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6716,21708,2018,11,Illinois,Food,Norovirus,NaN,Suspected,Caterer (food prepared off-site from where ser...,21,...,0.0,20.0,italian beef sandwich,NaN,Multiple,NaN,NaN,NaN,NaN,NaN
6717,21713,2018,2,Georgia,Food,Norovirus,NaN,Suspected,Caterer (food prepared off-site from where ser...,17,...,0.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6718,21773,2018,3,Washington,Food,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,11,...,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6719,21787,2018,11,Illinois,Food,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,7,...,0.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#geographic network 
    #connect cases that occur within x months and bordering state 
    
neighbors = pd.read_csv('neighbors-states.csv')
    


FileNotFoundError: [Errno 2] No such file or directory: 'neighbors-states.csv'

In [27]:
os.listdir()

['.git',
 '.ipynb_checkpoints',
 'BrynnyddHamilton_HW1.pdf',
 'BrynnyddHamilton_HW2.pdf',
 'brynn_cheats.ipynb',
 'class11_evolving_networks.pptx',
 'final_hw2_network.pdf',
 'hands on 2',
 'hands_on_exercise',
 'hands_on_exercise.zip',
 'Homework1.html',
 'Homework1.ipynb',
 'homework1.py',
 'homework2.py',
 'Homework2_Problem1.pdf',
 'Homework2_Problem3-4.pdf',
 'largest_component.csv',
 'List-of-neighboring-states-for-each-US-state-master',
 'List-of-neighboring-states-for-each-US-state-master.zip',
 'NationalOutbreakPublicDataTool - Copy.xlsx',
 'NationalOutbreakPublicDataTool.xlsx',
 'netscience',
 'netscience.zip',
 'Phys5116_Assignment1_2020.pdf',
 'Phys5116_Assignment2_2020.pdf',
 'project',
 'Untitled.pdf']